**Group 007-13 Project Proposal**

**Introduction**
The data set we chose is a tennis data set. We chose to keep specific columns in order to remove irrelevant data and keep the most important columns. We decided to keep if they are right or left handed, their current ranking, the player's name, their backhand throw, their current elo rank, and their prize money. A player's current elo rank is an estimate of their strength and is affected by their wins and loses. We are wondering whether a players bankhand throw and whether they are left or right handed affects their overall success. We will evaluate this by using their backhand, handedness, current rank, and current elo rank. 

**Preliminary exploratory data analysis**
*In code cells below, demonstrate that data can be read into R and then follow the instructions on Canvas*

In [9]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

download.file("https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS", "tennis_data.csv")
tennis_data <- read_csv("tennis_data.csv") |> 
    rename_all(make.names) |>  
    rename_with(str_to_lower) |>
    mutate(backhand = as_factor(backhand), handedness = as_factor(plays)) |>
    select(handedness, current.rank, name, backhand, current.elo.rank, prize.money, country)
tennis_data

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


handedness,current.rank,name,backhand,current.elo.rank,prize.money
<fct>,<chr>,<chr>,<fct>,<chr>,<chr>
Right-handed,378 (97),Oscar Jose Gutierrez,NA,NA,NA
Left-handed,326 (119),Jack Draper,Two-handed,NA,"$59,040"
Right-handed,178 (280),Lukas Lacko,Two-handed,144 (1764),"US$3,261,567"
⋮,⋮,⋮,⋮,⋮,⋮
Left-handed,495 (60),Gijs Brouwer,NA,NA,NA
NA,419 (81),Vladyslav Orlov,NA,NA,NA
Left-handed,451 (69),Aziz Dougaz,Two-handed,NA,"$61,984"


**Methods**
*Explain how you will conduct either your data analysis and which variables/columns you will use.
Describe at least one way that you will visualize the results*

**Expected outcomes and significance** *Answer questions on Canvas*